In [19]:
import pandas as pd
import json
def calculateAccuracy(priceData,jsonFile):
    monthlyPrice = pd.read_csv(priceData)
    monthlyPrice = monthlyPrice[['Date','Price']]
    monthlyPrice['Date'] = pd.to_datetime(monthlyPrice['Date'])
    monthlyPrice['changePrice'] = monthlyPrice['Price'].pct_change(fill_method ='ffill')
    with open(jsonFile) as json_file:
        data = json.load(json_file)
    date = []
    senti = []
    for key in data:
        date.append(key)
        senti.append(data[key])
    s1={'Date':date,'sentiment':senti}
    s1 = pd.DataFrame(s1)
    s1["Date"] = pd.to_datetime(s1['Date'])
    s1['changeSen'] = s1['sentiment'].pct_change(fill_method ='ffill')
    s1['changeSen'] = s1['changeSen'][::-1].shift(periods=1)
    new = s1.merge(monthlyPrice, on='Date')
    compare = new[['changeSen','changePrice']]
    compare = compare.dropna()
    compare['changeSen'] = [True if x > 0 else False for x in compare['changeSen']]
    compare['changePrice'] = [True if x > 0 else False for x in compare['changePrice']]
    compare['res'] = compare['changePrice'] == compare['changeSen']
    accuracy = compare['res'].mean()
    return accuracy